## Общее описание проекта##
Этот проект является чем-то вроде приложения-советчика для игрока в Dota 2 -- командную онлайн-игру 5 на 5, где каждый игрок в начале матча выбирает героя, на котором он будет играть. Программа **собирает последнюю статистику** по запрошенному персонажу (их в игре около 120) -- последние 500 матчей, сыгранных в соревновательном режиме игроками высокого уровня на этом герое, **обрабатывает** их, а затем выдает то, ради чего программа была написана -- список героев которые лучше всего себя показывают в одной команде с выбранным (то есть те, кого было бы эффективно взять **"в связке"** вместе с ним), а так же тех, кто луше всего справлялся с выбранным героем (как говорят на сленге дотеров, **"контрпики"**).

После этого программа анализирует то, как связанна продолжительность матча с шансом выбранного героя победить. Эта информация, бесполезная с точки зрения стороннего человека, на самом деле является крайне важной, так как она показывает, на какую стадию игры - начальную, среднюю или позднюю - приходится **"пик силы"** героя, то есть когда он раскрывается лучге всего, позволяя правильно построить план на игру. Эта информация наглядно предоставляется пользователю в виде графиков "распределения" процента побед и количества сыгранных и выигранных матчей на каждой стадии игры.

На этом основная часть программы заканчивается и следует что-то вроде бонуса для тех, кто дошел до конца. Во-первых, можно получить доступ к **расширенной информации** о том, как выбранный герой чувствует себя за и против других.

Во-вторых, можно посмотреть на результаты небольшого исследования. Оно заключается в том, что программа **исследует гипотезу** о том, что чем раньше было сделано первое убийство в игре, тем быстрее она закончится (логика в том, что чем агрессивнее играет одна из комманд, тем быстрее она либо захватит преимущетсво и закончит игру, либо, рискнув, потерпит поражение). Результаты, опять же, представлены на графике.

В-третьих, можно **включить музыку** в награду за терпение:)


## Руководство и комментарии для проверяющих##

**Прочитайте, пожалуйста, прежде чем запускать какие-либо ячейки!**

1) Программа для корректного выполнения требует, чтобы на компьютере были установлены **Selenium** с драйвером для **Google Chrome**-a по адресу 'C:\Program Files\chromedriver.exe' (ну и сам браузер), а так же библиотека **plotly**. Установите, пожалуйста!

2) В архиве помимо файла программы лежит текстовый файл, необходимый для ее работы -- он должен лежать с ней в одной папке.

3) Перед работой с программой сначала запустите все ячейки, начиная с той, где что-то импортируется, а потом уже запускайте main() из верхней ячейки.

4) На входе просят ввести id героя -- если вам нужно просто посмотреть на работу программы, то можно ввести почти любое число от 1 до 100, иначе можно глянуть в файл 'Hero Dictionary.txt', который появится в папке с прграммой сразу после запуска main()

5) Придется немного подождать - заранее спасибо!

6) В конце будет три кнопочки -- они кликабельны, предлагаю попробовать понажимать (только после каждого раза придется немного промотать экран вверх обратно на них - прошу прощения!)

7) Не открывайте окно хрома, пока не заиграет музыка, если выкинуло ошибку, скорее всего такую песню не нашло. Рекоммендуется вводить маленькими буквами без знаков препинания, так как песня в точности называется на яндекс музыке.

7) На всякий случай, перечислю то, что используется в моей программе:

 *Работа с API с динамическими ключами -- через него парсим матчи
 
 *Работа с selenium и библитотекой Action Chains -- включает музыку
 
 *Работа с pandas(довольно сложная) -- обработка всей статистики и извлечение данных.
 
 *Визуализация данных с помощью plotly
 
 *Математика - для регрессий и корреляций
 
 *Другие технологии - регулярные выражения для работы с текстовым файлом, продвинутые виждеты
 
 Размер кода по моим подсчетам -- 200+ строк.
 
 
 ## Большое спасибо за внимание!##

In [ ]:
main()

Please, enter the id of the hero that you want to check on. All ids can be found in the Hero Dictionary.
75
You've chosen Silencer. Now the programm will need some time to get data on the latest matches played.
Please, wait, it'll probably take about 3-5 minutes.


Output()

In [1]:
import re
import requests
from bs4 import BeautifulSoup
from IPython.display import clear_output
from IPython.display import display
from ipywidgets import Output
import ipywidgets as widgets
import pandas as pd
import time
import numpy as np
import math
import plotly as py
import plotly.graph_objs as go
import scipy.stats as st
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [4]:
def dictionary():
    file = open('npc_heroes.txt')
    out = open('Hero Dictionary.txt', 'w')
    names = []
    ids = []
    for x in file:
        if re.match(r'"npc_dota_hero', x.strip()) != None:
            names.append(re.split(r'[_,"]', x.strip(), maxsplit=4)[4][:-1])
            if re.split(r'[_,"]', x)[4] != 'base':
                out.write(re.split(r'[_,"]', x.strip(), maxsplit=4)[4].replace("_", " ").capitalize()+ ": ")
        if re.match(r'"HeroID', x.strip()) != None:
            ids.append(int(re.search(r'\d{1,}', x).group(0)))
            out.write(re.search(r'\d{1,}', x).group(0)+'\n')
    hero_dict = dict(zip(ids, names[1:]))
    return hero_dict
hero_dict=dictionary()

In [1]:
def get_ids(hero_id):
    url1 = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/'
    params1 = {
        'key' : 'EF2AC643577751B5AA751FFC10E7E051', 
        'skill' : '3', 
        'hero_id' : hero_id
    }
    count = 0.
    ids = []
    for i in range(5):
        r = requests.get(url1, params = params1)
        s = r.json()
        for x in (s['result']['matches']):
            ids.append(x['match_id'])
            params1['start_at_match_id'] = ids[-1]
    return ids

In [9]:
def get_match_details(ids):
    url2 = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/V001/'
    matches = []
    c = 0.
    out = Output()
    display(out)
    for x in ids:
        with out:
            display(widgets.FloatProgress(
                value=c/5,
                min=0,
                max=100,
                description='Loading:',
                bar_style='info',
                orientation='horizontal'
            ))
        params2 = {
            'match_id' : x,
            'key' : 'EF2AC643577751B5AA751FFC10E7E051'
        }
        r = requests.get(url2, params = params2).json()['result']
        c+=1
        matches.append(r)
        out.clear_output(wait=True)
    return matches

In [10]:
def get_results(matches, hero_id):
    results = []
    for r in matches:
        heroes = []
        winner = ""
        rad_vic = r['radiant_win']
        if r['game_mode'] == 22:
            for i in r['players']:
                heroes.append(hero_dict[(i['hero_id'])])
            chosen = heroes.index(hero_dict[hero_id])
            duration = float(r['duration'])/60
            fbtime = float(r['first_blood_time'])/60
            results.append([heroes, rad_vic, chosen, duration, fbtime])
    return results

In [11]:
def get_frame(results):
    frame = pd.DataFrame(results, columns = ['Heroes', 'Radiant Victory', 'Chosen', 'Duration', 'First Blood Time'])
    frame['Chosen Vic'] = (frame['Chosen'] < 5) == frame['Radiant Victory']
    return frame

In [12]:
def winrate(frame):
    ser = frame['Chosen Vic']
    try:
        winr = float(ser.tolist().count(True))/len(ser) * 100
    except ZeroDivisionError:
        return None
    return winr

In [13]:
def f(i, ally_id):
    for x in i:
        if x == hero_dict[ally_id]:
            return i.index(hero_dict[ally_id])
    return None
def together(a, b):
    rad = [0, 1, 2, 3, 4]
    dire = [5, 6, 7, 8, 9]
    return (a in rad and b in rad) or (a in dire and b in dire)
def against(a, b):
    rad = [0, 1, 2, 3, 4]
    dire = [5, 6, 7, 8, 9]
    return (a in rad and b in dire) or (a in dire and b in rad)
def winrate_with(frame, ally_id):
    df = frame.copy()
    picks = df['Heroes'].tolist()
    allyslot = [f(i, ally_id) for i in picks]
    heroslot = df['Chosen'].tolist()
    t = [together(i[0], i[1]) for i in list(zip(heroslot, allyslot))]
    df['together'] = t
    try:
        winrate_with = 100.0*list(df[df['together']==True]['Chosen Vic']).count(True)/t.count(True)
    except ZeroDivisionError:
        winrate_with = None
    return ([t.count(True), winrate_with])
def winrate_vs(frame, counter_id):
    df = frame.copy()
    picks = df['Heroes'].tolist()
    counterslot = [f(i, counter_id) for i in picks]
    heroslot = df['Chosen'].tolist()
    t = [against(i[0], i[1]) for i in list(zip(heroslot, counterslot))]
    df['against'] = t
    try:
        winrate_vs = 100.0*list(df[df['against']==True]['Chosen Vic']).count(True)/t.count(True)
    except ZeroDivisionError:
        winrate_vs = None
    return ([t.count(True), winrate_vs])

In [14]:
def get_advisor(frame, hero_id):
    ao = []
    for i in hero_dict.keys():
        b = winrate_with(frame, i)
        c = winrate_vs(frame, i)
        ao.append([b[0], b[1], c[0], c[1]])
    out1 = pd.DataFrame(ao, index = hero_dict.values(), columns = ["Games With", "Winrate With", 'Games Vs', 'Winrate Vs'])
    out1.drop(hero_dict[hero_id], inplace = True)
    out_with = out1.sort_values(by = ['Winrate With'], ascending = False)
    adv_with = out_with[out_with['Games With']>20][['Winrate With']][0:5].to_dict()['Winrate With']
    out_vs = out1.sort_values(by = ['Winrate Vs'])
    adv_vs = out_vs[out_vs['Games Vs']>20][['Winrate Vs']][0:5].to_dict()['Winrate Vs']
    return (adv_with, adv_vs, out1)

In [15]:
def statistic_during(frame, a, b=math.inf):
    frame_during = frame[(frame['Duration'] >= a) & (frame['Duration'] < b)]
    wins = list(frame_during['Chosen Vic']).count(True)
    games = len(list(frame_during['Chosen Vic']))
    return [games, wins, winrate(frame_during)]
def index(scale):
    index = []
    for x in range(len(scale)):
        try:
            index.append(str(scale[x])+' - '+str(scale[x+1])+' min')
        except IndexError:
            index.append(str(scale[x])+'+ min')
    return index
def get_distrib(frame):
    scale = [0, 15, 25, 30, 35, 45]
    ind = index(scale)
    distrib = []
    for i in range(len(scale)):
        try:
            distrib.append(statistic_during(frame, scale[i], scale[i+1]))
        except IndexError:
            distrib.append(statistic_during(frame, scale[i]))
    distrib = pd.DataFrame(distrib, index = ind, columns = ['Games', 'Wins', 'Winrate'])
    return distrib

In [16]:
def draw_distributions(distrib):
    py.tools.set_credentials_file(username='lutiy_luntik', api_key='zuMLMExH5jhjAgjPXSAE')
    a = go.Scatter(x = list(distrib.index), y = distrib['Winrate'])
    py.offline.init_notebook_mode(connected=True)
    layout = go.Layout( 
        title = 'Winrate Distribution'
    )
    fig = go.Figure(data = [a], layout = layout)
    py.offline.iplot(fig, filename='Winrate Distribution')
    trace1 = go.Bar(
        x = list(distrib.index),
        y = distrib['Wins'],
        name='Wins'
    )
    trace2 = go.Bar(
        x = list(distrib.index),
        y = distrib['Games'],
        name='Games'
    )

    data = [trace1, trace2]
    layout = go.Layout(
        barmode='group', 
        title = 'Games and Wins Distribution'
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig, filename='Wins Distribution')

In [17]:
def draw_regression(frame):
    fb_w = frame[frame['Chosen Vic']==True]['First Blood Time']
    dur_w = frame[frame['Chosen Vic']==True]['Duration']
    fb_l = frame[frame['Chosen Vic']==False]['First Blood Time']
    dur_l = frame[frame['Chosen Vic']==False]['Duration']
    reg = st.linregress(frame['First Blood Time'], frame['Duration'])
    trace0 = go.Scatter(
        x = fb_w,
        y = dur_w,
        mode = 'markers',
        name = 'Win',
        marker = dict(
            size = 8,
            color = 'rgb(0, 181, 56)',
            line = dict(
                width = 1, 
                color = 'rgb(0, 0, 0)'
            )
        )
    )
    trace1 = go.Scatter(
        x = fb_l,
        y = dur_l,
        mode = 'markers',
        name = 'Loss',
        marker = dict(
            size = 8,
            color = 'rgb(255, 14, 14)',
            line = dict(
                width = 1, 
                color = 'rgb(0, 0, 0)'
            )
        )
    )
    trace2 = go.Scatter(
        x = np.linspace(0, max(frame['First Blood Time'])), 
        y = np.linspace(0, max(frame['First Blood Time'])) * reg.slope + reg.intercept, 
        name = 'Linear Regression',
        line = dict(
            width = 6, 
            color = ('rgb(241, 17, 217)')
        )
    )
    data = [trace0, trace1, trace2]

    py.offline.iplot(data, filename='line-mode')
    return reg.rvalue

In [18]:
def dj_davai(muzyaka):
    import time
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys 
    browser = webdriver.Chrome('C:\Program Files\chromedriver.exe')
    browser.get('https://music.yandex.ru/home')
    time.sleep(5)
    un = browser.find_element_by_xpath('//*[@id="nb-1"]/body/div[1]/div[3]/div/div[1]/div[1]/div/div[1]/input')
    un.send_keys(muzyaka)
    time.sleep(1)
    from selenium.webdriver.common.action_chains import ActionChains
    actions = ActionChains(browser)
    actions.send_keys(Keys.DOWN, Keys.DOWN, Keys.ENTER).perform()
    time.sleep(1)
    actions.reset_actions()
    time.sleep(1)
    a = browser.find_element_by_class_name('d-track_selected')
    a.click()
    a.click()
    time.sleep(1)
    actions.move_to_element(a)
    actions.double_click()
    actions.perform()
    actions.reset_actions()

In [19]:
def main():
    hero_dict = dictionary()
    print('Please, enter the id of the hero that you want to check on. All ids can be found in the Hero Dictionary.')
    notok = True
    while notok:    
        try:
            hero_id = int(input())
            name = hero_dict[hero_id].replace("_", " ").capitalize()
            print('You\'ve chosen '+ name +'. Now the programm will need some time to get data on the latest matches played.\nPlease, wait, it\'ll probably take about 3-5 minutes.')
            notok = False
        except:
            print('There\'s no hero with such id. Please, enter a valid id.')
    ids = get_ids(hero_id)
    matches = get_match_details(ids)
    results = get_results(matches, hero_id)
    frame = get_frame(results)
    winr  = winrate(frame)
    print(name +' has '+str(round(winr, 2))+'% over the last 500 matches.')
    advisor = get_advisor(frame, hero_id)
    print("The best heroes to pick with " + name + " are:")
    for x in advisor[0].keys():
        print("\t > "+x.replace("_", " ").capitalize()+" with "+str(advisor[0][x])+"% winrate with "+name)
    print("\nThe best heroes to pick VS " + name + " are:")
    for x in advisor[1].keys():
        print("\t > "+x.replace("_", " ").capitalize()+" with "+str(100 - advisor[1][x])+"% winrate VS "+name)
    print('Here is some data on how strong is '+ name +' during different stages of the game:')
    distrib = get_distrib(frame)
    draw_distributions(distrib)
    
    button0 = widgets.Button(description="Full Data on Winrates", button_style='success')
    display(button0)
    def on_button0_clicked(b):
        print('*****************************************************************************')
        print(advisor[2])
        print('*****************************************************************************')
    button0.on_click(on_button0_clicked)
    
    button1 = widgets.Button(description="Interesting research:)", button_style='success')
    display(button1)
    def on_button1_clicked(b):
        print('*****************************************************************************')
        print('Here is some information of how the duration of the game depends on the timing of first blood:')
        a = draw_regression(frame)
        print('Correlation coefficient between duration and FB time is '+str(round(a, 2)))
        print('*****************************************************************************')
    button1.on_click(on_button1_clicked)
    
    button2 = widgets.Button(description="Play some music", button_style='success')
    display(button2)
    def on_button2_clicked(b):
        print('*****************************************************************************')
        print('What song do you want to listen to?')
        music_search = widgets.Text(
            value="",
            placeholder="Track name and singer",
            description="Search:",
            disabled=False
            )
        display(music_search)

        def handle_submit(sender):
            try:
                dj_davai(music_search.value)
            except:
                print('Something went wrong:( Please, click \'Play music\' button once more.')
            music_search.disabled = True

        music_search.on_submit(handle_submit)
        print('*****************************************************************************')
    button2.on_click(on_button2_clicked)
    return

In [7]:
dj_davai('наркотик кал')